In [9]:
import serial
import time

In [10]:
def unscramble(data, magic=0x88):
    rlt = []
    for i in data:
        a = (i + 0xFF) & 0xFF
        b = a ^ magic
        rlt.append((b & 0x7E) | ((b >> 7) & 0x01) | ((b << 7) & 0x80))
    return rlt

def scramble(data, magic=0x88):
    rlt = []
    for i in data:
        a = (i & 0x7E) | ((i >> 7) & 0x01) | ((i << 7) & 0x80)
        b = a ^ magic
        rlt.append((b + 1) & 0xFF)
    return rlt

def reset_serial(s):
    s.setDTR(False)
    s.setRTS(False)
    time.sleep(0.1)
    
def init_serial(s):
    s.setDTR(False)
    s.setRTS(True)
    time.sleep(0.005)
    s.setDTR(True)
    s.setRTS(False)

In [11]:
s = serial.Serial()
s.port = "/dev/ttyUSB0"
s.baudrate = 38400
s.parity = serial.PARITY_NONE
s.stopbits = serial.STOPBITS_ONE
s.timeout = 1
s.open()

In [46]:
reset_serial(s)
init_serial(s)

print("Listing files:")
file_index = 0
while True:
    s.write(scramble([0xe8, 0x01, 0x00, file_index]))
    reply = []
    while True:
        data = s.read()
        if not data:
            break
        reply.extend(unscramble(data))
    if not reply:
        break
    print(bytes(reply[4:]).decode("utf8"))
    file_index += 1

Listing files:
 
POGO 
BASE 
HOLES 
SLOT 
CAP 
UNNAMEDP 
UNNAMEDW 
UNNAMED 
TEST 
TEST 


In [27]:
reset_serial(s)
init_serial(s)
s.write(scramble([0xE8, 0x02, 0xE7, 0x01] + list(("TEST").encode("utf-8")) + [0x00]))
with open("repacked.rd", "rb") as f:
    s.write(f.read())

In [52]:
reset_serial(s)
init_serial(s)
s.write(scramble([0xe8, 0x03, 0x00, 0x02]))
s.write(scramble([0xe8, 0x04]))

reply = []
while True:
    data = s.read()
    if not data:
        break
    reply.extend(unscramble(data))
print(reply)

[]
